In [1]:
!pip3 install --upgrade pip

In [2]:
!pip3 install google-cloud-bigquery

In [3]:
!pip3 install pandas-gbq -U

In [4]:
!pip3 install db-dtypes

In [5]:
!pip3 install packaging --upgrade

In [6]:
# Import libraries
from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq
import os

print('Libraries imported successfully')

Libraries imported successfully


In [7]:
# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/c/Users/pavlos/OneDrive/Έγγραφα/script/turing-position-480316-e6-4c05555a07c9.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE

In [8]:
# Set your Google Cloud project ID and BigQuery dataset details

# -- YOUR CODE GOES BELOW THIS

project_id = 'turing-position-480316-e6' # Edit with your project id
dataset_id = 'staging_db' # Modify the necessary schema name: staging_db, reporting_db etc.
table_id = 'stg_film' # Modify the necessary table name: stg_customer, stg_city etc.

# -- YOUR CODE GOES ABOVE THIS LINE

In [9]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# -- YOUR CODE GOES BELOW THIS LINE

# Define your SQL query here
query = """
with base as (
  select *
  from `turing-position-480316-e6.postgres_productionpublic.film` --Your table path
  )

  , final as (
    select
        film_id
        , title as film_title
        , description as film_description
        , language_id as film_language_id
        , original_language_id as film_original_language_id
        , rental_duration as film_rental_duration
        , rental_rate as film_rental_rate
        , length as film_length
        , replacement_cost as film_replacement_cost
        , rating as film_rating
        , last_update as film_last_update
        , special_features as film_special_features
        , fulltext as film_fulltext
   FROM base
  )

  select * from final
"""

# -- YOUR CODE GOES ABOVE THIS LINE

# Execute the query and store the result in a dataframe
df = client.query(query).to_dataframe()

# Explore some records
df.head()

/home/pavlos/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,film_id,film_title,film_description,film_language_id,film_original_language_id,film_rental_duration,film_rental_rate,film_length,film_replacement_cost,film_rating,film_last_update,film_special_features,film_fulltext
0,638,OPERATION OPERATION,A Intrepid Character Study of a Man And a Fris...,1,<NA>,7,2.990000000,156,23.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers"",""Commentaries"",""Deleted Scenes"",""B...",'ancient':19 'charact':5 'china':20 'frisbe':1...
1,354,GHOST GROUNDHOG,A Brilliant Panorama of a Madman And a Compose...,1,<NA>,6,4.990000000,85,18.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers"",""Commentaries"",""Deleted Scenes"",""B...",'ancient':18 'brilliant':4 'car':16 'compos':1...
2,954,WAKE JAWS,A Beautiful Saga of a Feminist And a Composer ...,1,<NA>,7,4.990000000,73,18.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers"",""Commentaries"",""Deleted Scenes"",""B...",'beauti':4 'berlin':18 'challeng':14 'compos':...
3,216,DAY UNFAITHFUL,A Stunning Documentary of a Composer And a Mad...,1,<NA>,3,4.990000000,113,16.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers"",""Commentaries"",""Deleted Scenes"",""B...",'boat':23 'compos':8 'day':1 'documentari':5 '...
4,654,PANKY SUBMARINE,A Touching Documentary of a Dentist And a Sumo...,1,<NA>,4,4.990000000,93,19.990000000,G,2022-09-10 16:46:03.905795+00:00,"[""Trailers"",""Commentaries"",""Deleted Scenes"",""B...",'boy':17 'dentist':8 'documentari':5 'gulf':20...


In [12]:
# Define the full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# -- YOUR CODE GOES BELOW THIS LINE
# Define table schema based on the project description

schema = [
    bigquery.SchemaField('film_id', 'INTEGER'),
    bigquery.SchemaField('film_title', 'STRING'),
    bigquery.SchemaField('film_description', 'STRING'),
    bigquery.SchemaField('film_language_id', 'INTEGER'),
    bigquery.SchemaField('film_original_language_id', 'INTEGER'),
    bigquery.SchemaField('film_rental_duration', 'INTEGER'),
    bigquery.SchemaField('film_rental_rate', 'NUMERIC'),
    bigquery.SchemaField('film_length', 'INTEGER'),
    bigquery.SchemaField('film_replacement_cost', 'NUMERIC'),
    bigquery.SchemaField('film_rating', 'STRING'),
    bigquery.SchemaField('film_last_update', 'DATETIME'),
    bigquery.SchemaField('film_special_features', 'STRING'),
    bigquery.SchemaField('film_fulltext', 'STRING')
    ]

# -- YOUR CODE GOES ABOVE THIS LINE

In [13]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# Check if the table exists
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except Exception:
        return False

# Write the dataframe to the table (overwrite if it exists, create if it doesn't)
if table_exists(client, full_table_id):
    # If the table exists, overwrite it
    destination_table = f"{dataset_id}.{table_id}"
    # Write the dataframe to the table (overwrite if it exists)
    to_gbq(df, destination_table, project_id=project_id, if_exists='replace')
    print(f"Table {full_table_id} exists. Overwritten.")
else:
    # If the table does not exist, create it
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"Table {full_table_id} did not exist. Created and data loaded.")

Table turing-position-480316-e6.staging_db.stg_film did not exist. Created and data loaded.


In [14]:
# Below line converts your i.pynb file to .py python executable file. Modify the input and output names based
# on the table you are processing.
# Example:
# ! jupyter nbconvert stg_customer.ipynb --to python

# -- YOUR CODE GOES BELOW THIS LINE

!python3 -m jupyter nbconvert stg_actor.ipynb --to python

# -- YOUR CODE GOES ABOVE THIS LINE

[NbConvertApp] Converting notebook stg_actor.ipynb to python
[NbConvertApp] Writing 3946 bytes to stg_actor.py
